# Data Analysis

In [1]:
import numpy as np
import pandas as pd


In [2]:
buy_offer_df = pd.read_csv('./machine_learning/buyOffer.csv')
buy_offer_df['timestamp'] = pd.to_datetime(buy_offer_df['timestamp'])
buy_offer_df[:2]

,amount,consumer_id,http_code,left_in_stock,merchant_id,offer_id,price,product_id,quality,timestamp,uid
0,1,KNy+2xA78m6RjFXjNuuCU/5J2c7s1+a3On2J/Te4QnI=,200,0,sN7jrROVR1hljMZ5OHSLG6cKTwAxKmqDO0OAtWql7Ms=,7507,47.0,2,1,2017-02-01 19:23:43.674,21
1,1,KNy+2xA78m6RjFXjNuuCU/5J2c7s1+a3On2J/Te4QnI=,200,0,sN7jrROVR1hljMZ5OHSLG6cKTwAxKmqDO0OAtWql7Ms=,7508,38.0,1,4,2017-02-01 19:23:46.225,14


In [3]:
ms_df = pd.read_csv('./machine_learning/marketSituation.csv')
ms_df['timestamp'] = pd.to_datetime(ms_df['timestamp'])
ms_df['timespan'] = ms_df['timestamp'].diff().shift(-1)
ms_df[:4]

,amount,merchant_id,offer_id,price,prime,product_id,quality,shipping_time_prime,shipping_time_standard,timestamp,uid,timespan
0,1,sN7jrROVR1hljMZ5OHSLG6cKTwAxKmqDO0OAtWql7Ms=,7421,38.0,True,3,4,1,5,2017-02-01 15:55:45.482,34,0 days
1,1,Is16KAXSx7U7whXnqhRVcJD+JVAFleqAeNdQN4WQoV8=,7412,47.0,True,3,1,1,1,2017-02-01 15:55:45.482,31,0 days
2,3,4MRZ5eoIHSGs1GvLio2XKGksUFZIlTOcdy24mbnwq5c=,7345,18.0,True,3,1,1,5,2017-02-01 15:55:45.482,31,0 days
3,1,hPjEe9kUnPadEcs0jO1HLUL5maZPb6umcWgcbCxHzdo=,7422,9.4,True,3,3,1,5,2017-02-01 15:55:45.482,33,0 days


In [52]:
def match_timestamps(continuous_timestamps, point_timestamps):
    t_ms = pd.DataFrame({
        'timestamp': continuous_timestamps,
        'origin': np.zeros((len(continuous_timestamps)))
    })
    t_bo = pd.DataFrame({
        'timestamp': point_timestamps,
        'origin': np.ones((len(point_timestamps)))
    })

    t_combined = pd.concat([t_ms, t_bo], axis=0).sort_values(by='timestamp')
    original_locs = t_combined['origin'] == 1

    t_combined.loc[original_locs, 'timestamp'] = np.nan
    while t_combined['timestamp'].isnull().any():
        t_combined.loc[original_locs, 'timestamp'] = t_combined['timestamp'].shift(1)[original_locs]

    return t_combined[original_locs]['timestamp']

In [61]:
buy_offer_df['timestamp'] = match_timestamps(ms_df['timestamp'], buy_offer_df['timestamp'])
buy_offer_df[:1]

,amount,consumer_id,http_code,left_in_stock,merchant_id,offer_id,price,product_id,quality,timestamp,uid
0,1,KNy+2xA78m6RjFXjNuuCU/5J2c7s1+a3On2J/Te4QnI=,200,0,sN7jrROVR1hljMZ5OHSLG6cKTwAxKmqDO0OAtWql7Ms=,7507,47.0,2,1,2017-02-01 19:23:43.567,21


In [4]:
ms_df_prod1 = ms_df[ms_df['product_id'] == 1]
ms_df_prod1[:2]

,amount,merchant_id,offer_id,price,prime,product_id,quality,shipping_time_prime,shipping_time_standard,timestamp,uid,timespan
18,1,sN7jrROVR1hljMZ5OHSLG6cKTwAxKmqDO0OAtWql7Ms=,7420,47.0,True,1,1,1,5,2017-02-01 15:55:45.514,11,0 days
19,1,XUxM+XmfkUI2ZeZ2Mphwf066k46Y17X5QaiqDioocZo=,7352,16.5,True,1,1,1,5,2017-02-01 15:55:45.514,11,0 days


In [81]:
# ms_df.groupby('timestamp')[]

merged_df = pd.merge(ms_df, buy_offer_df, on='timestamp', how='outer')
print(merged_df.columns)

g = merged_df.groupby('timestamp')
g['timestamp'].first()

Index(['amount_x', 'merchant_id_x', 'offer_id_x', 'price_x', 'prime',
       'product_id_x', 'quality_x', 'shipping_time_prime',
       'shipping_time_standard', 'timestamp', 'uid_x', 'timespan', 'amount_y',
       'consumer_id', 'http_code', 'left_in_stock', 'merchant_id_y',
       'offer_id_y', 'price_y', 'product_id_y', 'quality_y', 'uid_y'],
      dtype='object')


False

In [55]:
merchant_id = 'sN7jrROVR1hljMZ5OHSLG6cKTwAxKmqDO0OAtWql7Ms='
#own_ms_view = ms_df[ms_df['triggering_merchant_id'] == merchant_id]
own_ms_view = ms_df
own_sales = buy_offer_df[buy_offer_df['merchant_id'] == merchant_id]
own_sales.loc[:,'timestamp'] = match_timestamps(own_ms_view['timestamp'], own_sales['timestamp'])

data_products = {}
for product_id in np.unique(own_ms_view['product_id']):
    ms_df_prod = own_ms_view[own_ms_view['product_id'] == product_id]
    
    dict_array = []
    for timestamp, group in ms_df_prod.groupby('timestamp'):
        competitors = group[group['merchant_id'] != merchant_id]
        own_situation = group[group['merchant_id'] == merchant_id]
        has_offer = len(own_situation) > 0
        own_price = float(own_situation['price'].mean()) if has_offer else np.nan
        own_price_rank = (group['price'] < own_price).sum() + 1 if has_offer else np.nan

        dict_array.append({
            'timestamp': timestamp,
            'sold': own_sales[own_sales['timestamp'] == timestamp]['amount'].sum(),
            'own_price': own_price,
            'own_price_rank': own_price_rank,
            'cheapest_competitor': competitors['price'].min(),
            'best_competitor_quality': competitors['quality'].max(),
        })

    data_products[product_id] = pd.DataFrame(dict_array)

data_products[1]

/Users/mbornstein/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,best_competitor_quality,cheapest_competitor,own_price,own_price_rank,sold,timestamp
0,3.0,6.7,47.0,4.0,0,2017-02-01 15:55:45.514
1,3.0,6.7,47.0,4.0,0,2017-02-01 15:55:46.569
2,3.0,6.7,47.0,4.0,0,2017-02-01 15:55:47.623
3,3.0,6.7,47.0,4.0,0,2017-02-01 15:55:48.683
4,3.0,6.7,47.0,4.0,0,2017-02-01 15:55:49.745
5,3.0,6.7,47.0,4.0,0,2017-02-01 15:55:50.813
6,3.0,6.7,47.0,4.0,0,2017-02-01 15:55:51.872
7,3.0,6.7,47.0,4.0,0,2017-02-01 15:55:52.931
8,3.0,6.7,47.0,4.0,0,2017-02-01 15:55:53.983
9,3.0,6.7,47.0,4.0,0,2017-02-01 15:55:55.045


In [83]:
for product_id in data_products:
    data = data_products[product_id]
    data.to_csv('{}_final.csv'.format(product_id))

In [25]:
import sys
sys.path.append('./merchant_sdk/')

from merchant_sdk.api import KafkaApi

kafka_api = KafkaApi()

In [26]:
buy_offer_csv_url = kafka_api.request_csv_export_for_topic('buyOffer')
buy_offer_df = pd.read_csv(buy_offer_csv_url)
print(buy_offer_df.shape)
buy_offer_df[:5]

request <class 'merchant_sdk.api.KafkaApi.KafkaApi'> get export/data/buyOffer () {}
(468, 11)


,amount,consumer_id,http_code,left_in_stock,merchant_id,offer_id,price,product_id,quality,timestamp,uid
0,1,KNy+2xA78m6RjFXjNuuCU/5J2c7s1+a3On2J/Te4QnI=,200,0,sN7jrROVR1hljMZ5OHSLG6cKTwAxKmqDO0OAtWql7Ms=,7507,47.0,2,1,2017-02-01T19:23:43.674Z,21
1,1,KNy+2xA78m6RjFXjNuuCU/5J2c7s1+a3On2J/Te4QnI=,200,0,sN7jrROVR1hljMZ5OHSLG6cKTwAxKmqDO0OAtWql7Ms=,7508,38.0,1,4,2017-02-01T19:23:46.225Z,14
2,1,KNy+2xA78m6RjFXjNuuCU/5J2c7s1+a3On2J/Te4QnI=,200,0,XUxM+XmfkUI2ZeZ2Mphwf066k46Y17X5QaiqDioocZo=,7504,18.0,1,3,2017-02-01T19:23:54.886Z,13
3,1,KNy+2xA78m6RjFXjNuuCU/5J2c7s1+a3On2J/Te4QnI=,200,1,XUxM+XmfkUI2ZeZ2Mphwf066k46Y17X5QaiqDioocZo=,7502,30.0,2,1,2017-02-01T19:23:59.830Z,21
4,1,KNy+2xA78m6RjFXjNuuCU/5J2c7s1+a3On2J/Te4QnI=,200,0,XUxM+XmfkUI2ZeZ2Mphwf066k46Y17X5QaiqDioocZo=,7510,30.0,4,1,2017-02-01T19:24:09.641Z,41


In [27]:
market_situation_csv_url = kafka_api.request_csv_export_for_topic('marketSituation')
market_situation_df = pd.read_csv(market_situation_csv_url)
print(buy_offer_df.shape)
market_situation_df[:5]

request <class 'merchant_sdk.api.KafkaApi.KafkaApi'> get export/data/marketSituation () {}


KeyError: 'url'